# Otimização dos pesos com PSO

Neste notebook, vamos rodar um experimento para otimização dos pesos com o PSO, com o
objetivo de maximizar o NDCG@24 com os dados do REGIS.

> Este experimento necessita que as configurações do **MongoDB**, **Elasticsearch** e **AQE Python** estejam definidas no `.env`.

In [1]:
from dotenv import load_dotenv
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.model_selection import train_test_split

from utils.aqe_pso import AQEPSO

load_dotenv()

True

Primeiramente, vamos separar as queries do REGIS dataset em treino e teste de acordo
com a análise do [AQE Python](../htmls/aqe_python_analysis.html).
Vamos separar as queries de maneira estratificada baseado no NDCG, com objetivo de
manter a mesma distribuição dos NDCGs em treino e teste.

In [2]:
regis_ndcgs = pd.DataFrame({
    "query_id": [f"Q{i}" for i in range(1, 35)],
    "ndcg": [.7719, .9198, .7985, .7515, .5955, .6564, .9640, .9862, .9689, .9666, .5514, .8548, .9627, .9797, .4468, .3421, .8286, .8280, .6997, .7641, .7227, .9150, .9438, .8854, .9663, .7734, .9525, .9226, .8856, .8109, .5275, .8596, .6378, .6721]
}).assign(
    ndcg_bin = lambda row: (np.searchsorted(np.sort(row.ndcg), row.ndcg) / 4).astype(int)
)

X_train, X_test, y_train, y_test = train_test_split(
    regis_ndcgs.filter(items=["query_id"]),
    regis_ndcgs.filter(items=["ndcg"]),
    stratify=regis_ndcgs.filter(items=["ndcg_bin"]),
    test_size=0.25,
    random_state=1234
)

print(f"O NDCG@24 médio do treino é {y_train.ndcg.mean():.4f} e o do teste é {y_test.ndcg.mean():.4f}")

O NDCG@24 médio do treino é 0.7919 e o do teste é 0.8129


Vejamos a distribuição do NDCG@24 do treino:

In [3]:
fig = px.histogram(y_train, x="ndcg")
fig.show()

E agora o do teste:

In [4]:
fig = px.histogram(y_test, x="ndcg")
fig.show()

Podemos ver que ambas distribuições tem uma longa cauda a esquerda e tem um valor médio similar.

## Classe do PSO

Abaixo está definida a classe que manipula o PSO e interage com o AQE e Elasticsearch.

In [5]:
pso_handler = AQEPSO(
    params={
        "SYN": (0, 1),
        "BT": (0, 1),
        "NT": (0, 1),
        "SA": (0, 1),
        "age_of": (0, 1),
        "located_in": (0, 1),
        "crosses": (0, 1),
        "constituted_by": (0, 1),
        "has_age": (0, 1),

        "ANP_GLOSSARY": (0, 1),
        "ANP_WELL_NAMES": (0, 1),
        "ANP_WELL_TERMS": (0, 1),
        "BDIEP_INSTANCES_ASSET": (0, 1),
        "BDIEP_INSTANCES_FIELD": (0, 1),
        "BDIEP_WELL_NAMES": (0, 1),
        "CURATED_LIST": (0, 1),
        "DICTIONARY": (0, 1),
        "HANDMADE_LIST": (0, 1),
        "LIST_OF_ABREVIATIONS": (0, 1),
        "MWE_LIST": (0, 1),
        "NER_ONTOLOGIES": (0, 1),
        "NER_WELL_NAMES": (0, 1),
        "PARTEX_ACRONYMIS": (0, 1),
        "REGIS_THESAURUS": (0, 1),

        "defaultSimilarity": (0, 1), 
        "similarTermsDefaultValue": (0, 1),
        "defaultNumberOfExpansions": (0, 5),
        "skosWeight": (0, 1),
        "wordEmbeddingWeight": (0, 1),

        "max_expanded_terms": (0, 10),
    },
    relation_keys=[
        "SYN",
        "BT",
        "NT",
        "SA",
        "age_of",
        "located_in",
        "crosses",
        "constituted_by",
        "has_age",
    ],
    source_keys=[
        "ANP_GLOSSARY",
        "ANP_WELL_NAMES",
        "ANP_WELL_TERMS",
        "BDIEP_INSTANCES_ASSET",
        "BDIEP_INSTANCES_FIELD",
        "BDIEP_WELL_NAMES",
        "CURATED_LIST",
        "DICTIONARY",
        "HANDMADE_LIST",
        "LIST_OF_ABREVIATIONS",
        "MWE_LIST",
        "NER_ONTOLOGIES",
        "NER_WELL_NAMES",
        "PARTEX_ACRONYMIS",
        "REGIS_THESAURUS",
    ],
    we_keys=[
        "defaultSimilarity", 
        "similarTermsDefaultValue",
        "defaultNumberOfExpansions",
        "skosWeight",
        "wordEmbeddingWeight",
    ],
    train_queries=X_train.query_id.tolist(),
    test_queries=X_test.query_id.tolist()
)

best_ndcg, best_params = pso_handler.execute_optimizer(
    iterations=150,
    n_particles=50,
    options = {'c1': 1.496172, 'c2': 1.496172, 'w': 0.72984}
)

2023-07-18 12:14:07,247 - pyswarms.single.global_best - INFO - Optimize for 150 iters with {'c1': 1.496172, 'c2': 1.496172, 'w': 0.72984}
pyswarms.single.global_best: 100%|██████████|150/150, best_cost=-.802
2023-07-20 04:36:29,067 - pyswarms.single.global_best - INFO - Optimization finished | best cost: -0.801606150882797, best pos: [0.58153775 0.51302839 0.37480284 0.48657333 0.48143366 0.59703546
 0.07190282 0.55358886 0.63740545 0.55630914 0.80046826 0.24071268
 0.550962   0.56076982 0.39848444 0.63817913 0.20023575 0.64118506
 0.91508646 0.71100726 0.68204754 0.45483497 1.90953837 0.33871548
 0.28438329 0.45691786 5.39215671 0.15231695 0.33478773 0.39352373]


In [6]:
print(f"Achieved an NDCG@24 of {best_ndcg} with the following parameters: {best_params}")

Achieved an NDCG@24 of 0.801606150882797 with the following parameters: {'ANP_GLOSSARY': 0.5815377482725479, 'ANP_WELL_NAMES': 0.5130283855552437, 'ANP_WELL_TERMS': 0.3748028357610644, 'BDIEP_INSTANCES_ASSET': 0.4865733325733583, 'BDIEP_INSTANCES_FIELD': 0.48143365812148753, 'BDIEP_WELL_NAMES': 0.5970354615617391, 'BT': 0.07190282406379467, 'CURATED_LIST': 0.5535888580402436, 'DICTIONARY': 0.6374054549535964, 'HANDMADE_LIST': 0.556309139578159, 'LIST_OF_ABREVIATIONS': 0.8004682587789169, 'MWE_LIST': 0.24071268105048235, 'NER_ONTOLOGIES': 0.5509620009677048, 'NER_WELL_NAMES': 0.5607698217760837, 'NT': 0.3984844390877025, 'PARTEX_ACRONYMIS': 0.6381791253129067, 'REGIS_THESAURUS': 0.2002357474342411, 'SA': 0.6411850602742958, 'SYN': 0.9150864583622139, 'age_of': 0.7110072620424651, 'constituted_by': 0.6820475429029835, 'crosses': 0.4548349735760693, 'defaultNumberOfExpansions': 1.909538367308422, 'defaultSimilarity': 0.33871548192774587, 'has_age': 0.2843832898229641, 'located_in': 0.4569

Podemos ver que o NDCG@24 obtido no treino foi de 80,16%, que é muito próximo ao que possuíamos antes da otimização, que obteve 81,31%, a qual não possuia o dataset NER_ONTOLOGIES, o qual incluiu sete novas relações, como possuia o dataset BDIEP_INSTANCES_ASSET e BDIEP_INSTANCES_FIELD unificado em um só.

## Conclusão

Os parâmetros obtidos com o experimento de PSO trouxeram um pequeno ganho comparado a antes da otimização, onde antes o NDCG@24 era de 79,19% e depois da otimização passou para 80,16%.
Os parâmetros obtigos podem ser utilizados como base para ajustes manuais mais finos.